In [ ]:
import json
import requests
import os
import base64
import sklearn

from eval_cus import check
import numpy as np
from sklearn.metrics.cluster import normalized_mutual_info_score as nmi
import torch


In [ ]:
url = "https://api-us.faceplusplus.com/imagepp/v1/facealbum/createalbum"
secret = "" 
key = ""

In [ ]:
k = requests.post(url, data={"api_key": key, "api_secret": secret})
print(k.text
     )

In [ ]:

face_token = json.loads(k.text)["facealbum_token"]

In [ ]:
#add image
url = "https://api-us.faceplusplus.com/imagepp/v1/facealbum/addimage"
path = "custom3"
imageid2ground = {}
images_stored = []
no_faces = []
faces = []

persons = os.listdir(path)
ground = 0
for g, person in enumerate(persons):
    if person.startswith('.'):
        continue
    
#     print(ground, person)
    co = 0
    images = os.listdir(path + '/' + person)
    print(f'curr:{ground} person:{person}')
    for image in images:
        print(f'curr:{ground} person:{person} co:{co}')
        if co >= 20:
            break
        with open(path + '/' + person + '/'+ image, "rb") as img_file:
            send_img64 = base64.b64encode(img_file.read())
        response = requests.post(url, data={"api_key": key, "api_secret": secret, "facealbum_token": face_token, "image_base64":send_img64})
#         print(response.text)
        if not response.ok:
            print(response.text)
            no_faces.append(path + '/' + person + '/'+ image)
            continue
        else:
            faces.append(path + '/' + person + '/'+ image)
            
        imgid = json.loads(response.text)["image_id"]
        imageid2ground[imgid] = ground
        images_stored.append(str(path + '/' + person + '/'+ image))
        co += 1
    ground += 1

In [ ]:

imageid2ground

In [ ]:
len(imageid2ground
   )

In [ ]:
np.save('images_stored.npy', images_stored)


In [ ]:
print(len(imageid2ground))

In [ ]:
print(len(images_stored))

In [ ]:
url = "https://api-us.faceplusplus.com/imagepp/v1/facealbum/getalbumdetail"
response = requests.post(url, data={"api_key": key, "api_secret": secret, "facealbum_token": face_token})
print(response.text)

In [ ]:
url = "https://api-us.faceplusplus.com/imagepp/v1/facealbum/groupface"
response = requests.post(url, data={"api_key": key, "api_secret": secret, "facealbum_token": face_token})
print(response.text)


In [ ]:
task_id = json.loads(response.text)["task_id"]

In [ ]:
url = "https://api-us.faceplusplus.com/imagepp/v1/facealbum/getalbumdetail"
response = requests.post(url, data={"api_key": key, "api_secret": secret, "facealbum_token": face_token, "task_id": task_id})
print(response.text)

In [ ]:
start_token = ''
predicted_clusters = []
ground_truth = []
url = "https://api-us.faceplusplus.com/imagepp/v1/facealbum/getalbumdetail"

while(True):
    if start_token:
        response = requests.post(url, data={"api_key": key, "api_secret": secret, "facealbum_token": face_token, "task_id": task_id, "start_token": start_token})
    else:
        response = requests.post(url, data={"api_key": key, "api_secret": secret, "facealbum_token": face_token, "task_id": task_id})
        
    print(len(json.loads(response.text)["faces"]))
    
    for j in json.loads(response.text)["faces"]:
        predicted_clusters.append(int(j["group_id"]))
        if j["group_id"] == -1 or j["group_id"] == '-1':
            print(j)
        ground_truth.append(imageid2ground[j["image_id"]])
        
    try:
        start_token = json.loads(response.text)["next_token"]
    except:
        print(response.text)
        break
        


In [ ]:
# predicted_clusters = []
# ground_truth = []

# print(len(json.loads(response.text)["faces"]))
# for j in json.loads(response.text)["faces"]:
#     predicted_clusters.append(int(j["group_id"]))
#     ground_truth.append(imageid2ground[j["image_id"]])

In [ ]:
# url = "https://api-us.faceplusplus.com/imagepp/v1/facealbum/getalbumdetail"
# start_token = 0
# predicted_clusters = []
# ground_truth = []

# while True:
#     if start_token:
#         response = requests.post(url, data={"api_key": key, "api_secret": secret, "facealbum_token": face_token, "task_id": task_id, "start_token":start_token})
#         try:
#             json.loads(response.text)["next_token"]
#         except:
#             print(response.text)
#             break
#     else:
#         response = requests.post(url, data={"api_key": key, "api_secret": secret, "facealbum_token": face_token, "task_id": task_id})
    
# #     start_token = json.loads(response.text)["next_token"]
#     for j in json.loads(response.text)["faces"]:
#         predicted_clusters.append(int(j["group_id"]))
#         ground_truth.append(imageid2ground[j["image_id"]])
    
# # print(response.text)

In [ ]:
print(len(predicted_clusters))


In [ ]:
print(len(persons))

In [ ]:
print(len(ground_truth))


In [ ]:
# clas = len(np.unique(ground_truth))
# ground_truth = torch.tensor(ground_truth).cuda()
# predicted_clusters = torch.tensor(predicted_clusters).cuda()

# p = check(ground_truth, predicted_clusters, num_classes=clas, class_names = [0]*clas)
# print(p)

In [ ]:
# print(nmi(ground_truth, predicted_clusters))

In [ ]:
print(np.unique(ground_truth))

In [ ]:
print(np.unique(predicted_clusters))

In [ ]:
# zip(ground_truth, predicted_clusters)

In [ ]:
from sklearn.metrics import normalized_mutual_info_score, adjusted_rand_score
# from scipy.optimize import linear_sum_assignment as linear_assignment
from sklearn.utils.linear_assignment_ import linear_assignment
import pandas as pd

def nmi(y_true, y_pred):
    return normalized_mutual_info_score(y_true, y_pred)


def ari(y_true, y_pred):
    return adjusted_rand_score(y_true, y_pred)


def acc(y_true, y_pred):
    y_true, y_pred = pd.Series(list(y_true)), pd.Series(list(y_pred))
    y_true = y_true.astype(np.int64)
    assert y_pred.size == y_true.size
    D = max(y_pred.max(), y_true.max()) + 1
    w = np.zeros((D, D), dtype=np.int64)
    for i in range(y_pred.size):
        w[y_pred[i], y_true[i]] += 1
    ind = linear_assignment(w.max() - w)
    return sum([w[i, j] for i, j in ind]) * 1.0 / y_pred.size

In [ ]:
acc(ground_truth, predicted_clusters)

In [ ]:
nmi(ground_truth, predicted_clusters)

In [ ]:
ari(ground_truth, predicted_clusters)